# Test Benchmark

## Buffer

buffer store function (multi done scenario)

In [ ]:
from agent import AgentSAC
from attrdict import AttrDict
import torch
import yaml
from envs import toy

with open(f'configs/debug.yaml', "r") as stream:
  try:
    cfg = AttrDict(yaml.safe_load(stream))
  except yaml.YAMLError as exc:
    print(exc)
agent = AgentSAC(cfg)

In [ ]:
traj = agent.explore_vec_env()

In [ ]:
agent.buffer.data_parser(agent.buffer.data[8:12], 'info.ag')

In [ ]:
trans = agent.buffer.sample_batch(10, her_rate=1, indices=torch.arange(11, device='cuda:0'))

In [ ]:
obs_dict = agent.EP.obs_parser(trans.state[:10])
info_dict = agent.EP.info_parser(trans.info[:10])
obs_dict.ag, obs_dict.g, info_dict.step, trans.rew

In [ ]:
info_dict = agent.EP.info_parser(trans.info[:20])
info_dict.step

HER buffer

# Wandb

In [ ]:
import wandb
wandb.init(project="debug")

# How to Check Block Distance

In [ ]:
import torch
num_envs=2
num_goals=3
min_dist = 1
extra_goals = torch.tensor(
  [
    [
      [[0,0],[0.5,0],[1.5,0]],
      [[0,0],[0.5,0.5],[1.5,0.5]]
    ],
    [
      [[0,0],[1.5,0],[0,1.5]],
      [[3,0],[0,0],[5,0]]
    ],
  ]
)
goal_dist = torch.abs(extra_goals.unsqueeze(-3) - extra_goals.unsqueeze(-2))
satisfied_idx = ((goal_dist[...,0] > min_dist) | \
  (goal_dist[..., 1] > min_dist)) | torch.eye(num_goals).bool()
satisfied_idx = satisfied_idx.all(dim=-1).all(dim=-1)
extra_goals[satisfied_idx].shape

In [ ]:
p=0.2
n=3
import math
import numpy as np
np.cumsum([math.factorial(n)/(math.factorial(n-m)*math.factorial(m))*(1-p)**m*p**(n-m) for m in range(n+1)])

# Rotate Experience

In [ ]:
import torch

grip_pos = torch.tensor([-1,-2,1,-2,2,1])
hand_vel = torch.tensor([0.1,0.1,-0.1,-0.1,-0.1,0.1])
finger_width = torch.tensor([0.01,0.02])
block_state = torch.tensor([0]*20)
block_pos = torch.tensor([-10,-20,10,10,20,-10])
goal = torch.tensor([1,2,3,4,5,6])

# obs = torch.cat((grip_pos, hand_vel, finger_width, block_state, block_pos, goal))

# pos_rot_mat = torch.tensor([
#   [-1,0,0],
#   [0,-1,0],
#   [0,0,1]
# ])
# quat_rot_mat = torch.tensor([
#   [0,-1,0,0],
#   [1,0,0,0],
#   [0,0,0,1],
#   [0,0,1,0],
# ])
# block_other_mat = torch.block_diag(*([quat_rot_mat]+[pos_rot_mat]*2))
# rot_mat = torch.block_diag(*([pos_rot_mat]*4+[torch.eye(4)]+[block_other_mat]*2+[pos_rot_mat]*4))
# print((obs*rot_mat)/obs)

In [ ]:
robot_obs = torch.cat((grip_pos, hand_vel, finger_width))
robot_reshape_mat = torch.tensor(
  [
    [1.,0,0]+[0]*11,
    [0,1,0]+[0]*11,
    [0,0,1]+[0]*11,
    [0]*7+[1,0,0]+[0]*4,
    [0]*7+[0,1,0]+[0]*4,
    [0]*7+[0,0,1]+[0]*4,
    [0]*3+[1,0,0]+[0]*8,
    [0]*3+[0,1,0]+[0]*8,
    [0]*3+[0,0,1]+[0]*8,
    [0]*10+[1,0,0]+[0],
    [0]*10+[0,1,0]+[0],
    [0]*10+[0,0,1]+[0],
    [0]*6+[1]+[0]*7,
    [0]*13+[1],
  ],
)
robot_obs@robot_reshape_mat

In [ ]:
# self attention
import torch
import torch.nn as nn
num_goals = 3
batch_size = 2
num_heads = 1
self_attn = nn.MultiheadAttention(4, num_heads)
ten_in = torch.randn(num_goals,batch_size,4)
mask = torch.tensor([[1,1,0], [1,0,0]]).bool()

In [ ]:
token, weight = self_attn(ten_in[[0]], ten_in, ten_in, ~mask)
token, weight

In [ ]:
token[mask.transpose(0, 1) == 0] = 0.0
token.sum(dim=0)/mask.sum(dim=1, keepdim=True)

# Wandb

In [1]:
import wandb
import numpy as np
api = wandb.Api()

In [ ]:
old_run = api.run("jc-bao/corl-main/1c2b7rak") #    1ml3z0m7 l3r9yo7c

In [ ]:
old_run_2 = api.run("jc-bao/corl-main/2ou8i9t4") #   25dqffwv klpwf6u6

In [ ]:
import json

for k, v in old_run.config.items():
  if isinstance(v, dict):
    for k2, v2 in v.items():
      if k2 in old_run_2.config[k]:
        if old_run_2.config[k][k2] != v2:
          print(k2, v2, old_run_2.config[k][k2])
      else:
        print(k2, v2)
  elif k in old_run_2.config:
    if old_run_2.config[k] != v:
      print(k, v, old_run_2.config[k])
  else:
    print(k, v)

In [ ]:
# old base
old_run_2 = api.run("jc-bao/corl-2022/19b0h9sh") #     syq3rcs8 1takfy4s

In [10]:
base_run = api.run("jc-bao/corl-2022/3q5t0kdk") #       2ou8i9t4 17ftkl7z 1xvkdusm
config = base_run.config
config['mix_robot'] = True
wandb.init(project="corl-2022", name="mix_robot", config = config); 
import math
for i, row in base_run.history().iterrows():
  row = row.to_dict()
  data = {}
  for k, v in row.items():
    if not math.isnan(v):
      data[k] = v
  if 'eval/final_rew' in data and 'curri/current_num_goals' in data:
    rew = data['eval/final_rew']
    num_goals = data['curri/current_num_goals']
    data['plot/arranged_block_number'] = num_goals * (rew+1)
  for key, value in row.items():
    wandb.log(data, step=int(row['_step']))
  # if row['_step'] > 3e7:
  #   break 
wandb.finish()

curri/current_num_goals,▁▁▁▁▁▁▁▂▃▄▅▆▇▇▇▇▇▇▇▇▇▇██████████████████
curri/os_rate,▁▁▁█████████████████████████████████████
curri/table_gap,▁▁▁▁▁▁▁▁▁▁▁▁▃▃▄▆████████████████████████
eval/curri.current_num_goals,▁▁▁▁▁▁▁▂▃▄▅▆▇▇▇▇▇▇▇▇▇▇██████████████████
eval/curri.os_rate,▁▁▁█████████████████████████████████████
eval/curri.table_gap,▁▁▁▁▁▁▁▁▁▁▁▁▃▃▄▆████████████████████████
eval/ep_rew,▅▅▄▅▆▆▇█▇▅▄▁▁▁▄▅▆▅▅▇▇▇█▇▇█▇█████▇███████
eval/ep_steps,▂▂▃▂▂▂▁▁▁▄▅██▇▄▃▃▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
eval/final_rew,▁▁▂▄▄▅▇█▇▇▆▆▆▅▅▆▆▄▅▇█▇██████████▇███████
eval/handover_0_1_success_rate,▁▂▃▆▇▇█████████████████████████
eval/handover_0_2_success_rate,▁▁▁▁▁▁▁▁▂▅▇█████████████████████████████


In [ ]:
# new base
old_run_2 = api.run("jc-bao/corl-main/26txi5ae") #  3p1rz5ys 2njs4v14 2v4xn8zm 

In [ ]:
wandb.init(project="corl-2022", name="random_base"); 

In [ ]:
import math
for i, row in old_run.history().iterrows():
  row = row.to_dict()
  data = {}
  for k, v in row.items():
    if not math.isnan(v):
      data[k] = v
  if 'eval/final_rew' in data and 'curri/current_num_goals' in data:
    rew = data['eval/final_rew']
    num_goals = data['curri/current_num_goals']
    data['plot/arranged_block_number'] = num_goals * (rew+1)
  for key, value in row.items():
    wandb.log(data, step=int(row['_step']))
  if row['_step'] > 3e7:
    break 

In [ ]:
last_step = int(row['_step']+1)

In [ ]:
import math
for i, row in old_run_2.history().iterrows():
  row = row.to_dict()
  data = {}
  for k, v in row.items():
    if not math.isnan(v):
      data[k] = v
  if 'eval/final_rew' in data and 'curri/current_num_goals' in data:
    rew = data['eval/final_rew']
    num_goals = data['curri/current_num_goals']
    data['plot/arranged_block_number'] = num_goals * (rew+1)
  for key, value in row.items():
    wandb.log(data, step=int(row['_step']))
  # if row['_step'] > 4.32e7:
  #   break

In [ ]:
wandb.finish();

# Export to jit

In [ ]:
import isaacgym
import torch
import wandb
from net import ActorFixSAC

cfg = torch.load('/home/pcy/rl/srl/outputs/2022-06-02/15-23-34/2ho_env.pkl', map_location='cuda:0')
save_path = wandb.restore('latest_17.pth', 'jc-bao/corl-2022/23bfj0ac').name
with open(save_path, 'rb') as f:
  checkpoint = torch.load(f, map_location='cuda:0')
act = ActorFixSAC(cfg)
act.load_state_dict(checkpoint["actor"], strict=False)
scripted_policy = torch.jit.script(act)
scripted_policy.save("scripted.pt")